In [22]:
from finviz.screener import Screener
import os
import datetime
import pandas as pd
import time
import yfinance as yf

In [23]:
def fetch_stocks():
    # filters = ['ind_exchangetradedfund','sh_avgvol_o2000']
    filters = ['sh_avgvol_o2000', 'sh_price_o20']
    stock_list = Screener(filters=filters, table='Performance', order='price')
    
    return stock_list

In [24]:
stock_list = fetch_stocks()

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:11<00:00,  2.70it/s]


In [25]:
def cal_VRRS(ticker, ref, vol_weight, rolling_length):
    ticker_sma = ticker.rolling(rolling_length).mean()
    ticker_change = ticker - ticker_sma
    
    ref_sma = ref.rolling(rolling_length).mean()
    ref_change = ref - ref_sma
    
    return (ticker_change/ticker_sma - ref_change/ref_sma) * vol_weight * 100

In [26]:
ticker_dict = {}

In [27]:
# Get data from Yahoo Finance for the S&P 500 and the VIX index:    
spy = yf.Ticker("SPY")
spy = spy.history(period="max")['Close']  

for stock in stock_list:
    
    ticker = stock['Ticker']

    stock = yf.Ticker(ticker)    
    stock = stock.history(period="max")['Close']  

    # Calculate the VRRS indicator:
    rolling_length = 21
    vol_weight = 1
    
    vrrs = cal_VRRS(stock, spy, vol_weight, rolling_length)
    
    ticker_dict[ticker] = vrrs.iloc[-1]
    
    print(ticker, vrrs.iloc[-1])

VET -2.9584418426926495
MGY -16.364609222178945
PINS 2.9695357924928825
ASAN 3.2101603702300876
IRT -5.049486745974906
BRX -2.393224471750329
DBA -4.770290623494812
TBF -1.638678816020402
ELAN -3.6856971913165433
BKLN -1.7900431677146726
UNG -13.837102720454261
CVET -1.5047969108485955
T 1.0223315171273135
BTU -9.124006813701028
GPK -0.09933334299819137
PATH 5.219765988715608
JWN -12.638089344614498
DUST 12.91104955325448
ELY -0.18578526877398616
EWA -2.6763061782280344
AES 2.416584339210241
STWD 0.4457615549650191
PHG -2.4396213111042973
NOVA 7.739037953574622
MRO -14.22973397610705
LAC 0.9010365971213963
SIX -7.872953819101988
FHN -1.3687498365180064
VXX -7.757180149628451
EXEL 7.611569056378616
EWH 0.1098413417957668
VST -6.734545971085682
MT -13.38976273792531
LBTYK -4.85872520477827
LTHM -8.851361296218508
MNDT 0.565317612090949
CCJ -2.739442150603473
UBER -1.5131292551846518
SBLK -12.347329792177597
TPX -0.6162551266678443
ILF -4.8814840512276065
NIO 5.421962138188019
EWG -6.0767

MCHP -2.420289987686598
CEG 0.24392432563068098
TRGP -7.325011039909279
HDB 5.379005629202889
SOXS -2.6004595942291946
COIN 7.247100907275342
OXY -0.5972375538128186
BLDR 7.116646047576896
PDD -3.4966105061956023
VGIT -1.3290339376529297
XRT -2.192068572391119
CSGP 2.419871700853368
JD -3.873705601173823
SPLV -0.34711490334785927
MDLZ 0.7259732939764101
PEG -1.6258876716545487
MET -2.176085541888664
FTNT 7.413728026924201
GILD 1.0940757714917786
STT -2.5683762144134072
KO 0.3977431240764079
GE -6.428931087438365
IVW 1.6699701732003944
SAIL 0.38574812673968334
COUP 4.5910533902167465
TD -5.296235158837509
SCHW 0.49059191446726125
ACC -1.2162282367487978
BAX -4.211469780012095
APH -0.6451089944675389
STLD -6.159915603501751
DOCU 0.498009762356897
IWR -0.8569227878208281
CTSH -3.731931474961843
SQ -0.24164909818652802
BJ 6.670620512908868
BALL -1.547059951737643
O 1.1375837322481683
XEL -0.3980518614780073
XLU -0.11721385616777283
OTIS -2.889429473369949
XLE -8.906078908111754
BBY -0.7672

In [28]:
sorted_x = sorted(ticker_dict.items(), key=lambda x: x[1])

In [33]:
for x in sorted_x:
    if abs(x[1]) > 10: 
        print(x[0], (round(x[1], 2)))

LABD -44.02
BOIL -33.08
UPST -25.43
KSS -24.29
TECK -22.26
SM -18.7
MUR -18.34
NUGT -18.0
ERX -17.41
PBF -16.96
FCX -16.66
MGY -16.36
UCO -15.57
SARK -15.17
AR -14.35
MRO -14.23
APA -14.17
UNG -13.84
RCL -13.58
MT -13.39
SQQQ -13.39
OVV -13.32
COP -13.11
XOP -12.96
GDXJ -12.93
HAL -12.85
FANG -12.84
CNQ -12.75
JWN -12.64
PSX -12.37
SBLK -12.35
EOG -12.32
EQT -12.05
SU -11.84
BBWI -11.66
TECS -11.42
MPC -10.78
DELL -10.7
RRC -10.53
BHP -10.44
VLO -10.44
FL -10.34
HES -10.28
PXD -10.26
RIO -10.21
MOS -10.13
MP -10.09
AA -10.07
TEAM 10.01
EXAS 10.37
OKTA 10.48
ETSY 11.37
RIVN 11.56
U 11.74
DUST 12.91
DOCS 13.18
XBI 13.66
AFRM 14.36
DRIP 14.46
ARKG 14.89
SNOW 15.25
EDU 16.24
GLBE 18.91
TDOC 19.67
MRNA 22.98
AGL 23.09
CFLT 23.69
BYND 23.88
RBLX 24.34
CHWY 27.88
NVAX 49.17
